In [1]:
SQL_SOURCE = 'https://tma353.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-77fe7a447ea3>, line 23)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
max_duration = 3 * 60 * 60
query = """
SELECT start_station_id, tripduration FROM citibike WHERE tripduration < {dur} ORDER BY start_station_id, tripduration
""".format(dur=max_duration)
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
max_duration = 3 * 60 * 60 # head and tail
query = """
SELECT start_station_id, tripduration FROM citibike WHERE tripduration < {dur} 
ORDER BY (start_station_id, tripduration) DESC LIMIT 10
""".format(dur=max_duration)
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',').head()

In [ ]:
max_duration = 3 * 60 * 60
query = """
SELECT start_station_id, tripduration FROM citibike WHERE tripduration < {dur} 
ORDER BY (start_station_id, tripduration) ASC LIMIT 10
""".format(dur=max_duration)
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',').head()

In [ ]:
max_duration = 3 * 60 * 60 # unique station IDs
query = """
SELECT DISTINCT start_station_id FROM citibike
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')


In [ ]:
max_duration = 3 * 60 * 60 # number of trips wc -I for SQL
query = """
SELECT COUNT(*) FROM citibike  
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
max_duration = 3 * 60 * 60 # min trip durartion 
query = """
SELECT MIN(tripduration) FROM citibike
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
max_duration = 3 * 60 * 60 # max trip duration
query = """
SELECT MAX(tripduration) FROM citibike
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
max_duration = 3 * 60 * 60
query = """
SELECT AVG(tripduration) FROM citibike
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
# trips Feb 2015 on weekends average trip duration, weekday?
query = """
SELECT * FROM citibike WHERE DATE(starttime)='2015-02-02'
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = """
SELECT to_char(starttime, 'Day'),* FROM citibike 
WHERE trim(to_char(starttime, 'Day'))='Saturday' OR trim(to_char(starttime, 'Day'))='Sunday'
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = """
SELECT AVG(tripduration) FROM citibike 
WHERE trim(to_char(starttime, 'Day'))='Saturday' OR trim(to_char(starttime, 'Day'))='Sunday'
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = """
SELECT AVG(tripduration) FROM citibike 
WHERE trim(to_char(starttime, 'Day'))!='Saturday' AND trim(to_char(starttime, 'Day'))!='Sunday'
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
# start station w/ GROUPBY within 500 meters 
# of Times Sqaure (40.7577, -73.9857)

In [ ]:
query = """
SELECT DISTINCT ON (start_station_id)
start_station_id,start_station_latitude,start_station_longitude FROM citibike 
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
query = """
SELECT COUNT(*) FROM citibike 
GROUP BY start_station_id
"""
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
times_sq =  "(40.7577,-73.9857)"
query = """
SELECT COUNT(*) FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
                 CDB_LatLng{times_sq}::geography,
                 500)
GROUP BY start_station_id
""".format(times_sq=times_sq)
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

In [ ]:
# longest average trip duration during weekends within 500 meters 
# of TimeSquare

In [ ]:
times_sq =  "(40.7577,-73.9857)"
query = """
SELECT * FROM (
    SELECT start_station_id, AVG(tripduration) as aver_trip FROM citibike 
    WHERE (trim(to_char(starttime, 'Day'))='Saturday' 
          OR trim(to_char(starttime, 'Day'))='Sunday')
          AND ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
                 CDB_LatLng{times_sq}::geography,
                 500)
    GROUP BY start_station_id ) as derivedtable ORDER BY derivedtable.aver_trip DESC LIMIT 1
""".format(times_sq=times_sq)
pd.read_csv(StringIO.StringIO(queryCartoDB(query)), sep=',')

## ASH Score 100. Extra CRedit 0